In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler

# Ler a planilha 'netflix_titles.csv'
df = pd.read_csv('netflix_titles.csv')

# Pré-processamento
# Converter dados categóricos para numéricos usando one-hot encoding
df_encoded = pd.get_dummies(df, columns=['type', 'country', 'rating', 'listed_in'])

# Normalizar os dados
scaler = StandardScaler()
df_encoded[['release_year']] = scaler.fit_transform(df_encoded[['release_year']])

# Separar os dados em variáveis de características e alvo
X = df_encoded.drop(columns=['type_Movie', 'type_TV Show'])
y = df_encoded['type_Movie']  # Supondo classificação binária para simplicidade

# Separar os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=42)

# Validação Cruzada K-Fold
kf = KFold(n_splits=5)
for train_index, val_index in kf.split(X_train):
    X_train_kf, X_val_kf = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]

print("Pré-processamento dos dados concluído. Pronto para a classificação SVM.")

Pré-processamento dos dados concluído. Pronto para a classificação SVM.


In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, learning_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Ler a planilha 'netflix_titles.csv'
df = pd.read_csv('netflix_titles.csv')

# Pré-processamento
# Remover colunas que não são necessárias para a classificação
df = df.drop(columns=['show_id', 'title', 'director', 'cast', 'date_added', 'description'])

# Remover a unidade de tempo 'min' da coluna 'duration' e converter para valores numéricos, lidando com valores nulos
df['duration'] = df['duration'].apply(lambda x: int(x.split()[0]) if isinstance(x, str) and 'min' in x else (int(x.split()[0]) * 60 if isinstance(x, str) else 0))

# Codificar a coluna 'type' para valores numéricos
label_encoder = LabelEncoder()
df['type'] = label_encoder.fit_transform(df['type'])

# Converter dados categóricos para numéricos usando one-hot encoding
df_encoded = pd.get_dummies(df, columns=['country', 'rating', 'listed_in'])

# Normalizar os dados
scaler = StandardScaler()
df_encoded[['release_year', 'duration']] = scaler.fit_transform(df_encoded[['release_year', 'duration']])

# Separar os dados em variáveis de características e alvo
X = df_encoded.drop(columns=['type'])
y = df_encoded['type']

# Separar os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

# Validação Cruzada K-Fold
kf = KFold(n_splits=5)
validation_reports = []
for train_index, val_index in kf.split(X_train):
    X_train_kf, X_val_kf = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]

    # Treinar o modelo utilizando SVM e kernel RBF
    svm_model = SVC(kernel='rbf')
    svm_model.fit(X_train_kf, y_train_kf)

    # Fazer previsões no conjunto de validação
    y_val_pred = svm_model.predict(X_val_kf)

    # Mostrar os resultados como Classification Report para o conjunto de validação
    report_val = classification_report(y_val_kf, y_val_pred, output_dict=True)
    validation_reports.append(report_val)

# Calcular a média dos relatórios de validação manualmente
avg_validation_report = {}
for key in validation_reports[0].keys():
    if isinstance(validation_reports[0][key], dict):
        avg_validation_report[key] = {}
        for sub_key in validation_reports[0][key].keys():
            avg_validation_report[key][sub_key] = sum(d[key][sub_key] for d in validation_reports) / len(validation_reports)
    else:
        avg_validation_report[key] = sum(d[key] for d in validation_reports) / len(validation_reports)

# Treinar o modelo final utilizando todo o conjunto de treino
svm_model_final = SVC(kernel='rbf')
svm_model_final.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_test_pred = svm_model_final.predict(X_test)

# Mostrar os resultados como Classification Report para o conjunto de teste
report_test = classification_report(y_test, y_test_pred, target_names=label_encoder.classes_)

accuracy = accuracy_score(y_test, y_test_pred)
print(f'Acurácia do modelo: {accuracy:.4f}')
#print("Validation Report (Average):\n", avg_validation_report)
print("Test Report:\n", report_test)

# Curvas de aprendizado
#train_sizes, train_scores, val_scores = learning_curve(svm_model_final, X_train, y_train, cv=kf, n_jobs=-1,
#                                                       train_sizes=np.linspace(0.1, 1.0, 10), scoring='accuracy')
#
#train_scores_mean = np.mean(train_scores, axis=1)
#val_scores_mean = np.mean(val_scores, axis=1)
#
#plt.figure()
#plt.plot(train_sizes, train_scores_mean, label='Acurácia no treino')
#plt.plot(train_sizes, val_scores_mean, label='Acurácia na validação')
#plt.xlabel('Tamanho do conjunto de treino')
#plt.ylabel('Acurácia')
#plt.title('Curvas de aprendizado')
#plt.legend()
#plt.grid(True)
#plt.show()

Acurácia do modelo: 0.9605
Test Report:
               precision    recall  f1-score   support

       Movie       0.96      0.99      0.97      5520
     TV Show       0.97      0.90      0.93      2407

    accuracy                           0.96      7927
   macro avg       0.96      0.94      0.95      7927
weighted avg       0.96      0.96      0.96      7927



In [32]:
import pygad
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

# Ler a planilha 'netflix_titles.csv'
df = pd.read_csv('netflix_titles.csv')

# Pré-processamento
# Remover colunas que não são necessárias para a classificação
df = df.drop(columns=['show_id', 'title', 'director', 'cast', 'date_added', 'description'])

# Remover a unidade de tempo 'min' da coluna 'duration' e converter para valores numéricos, lidando com valores nulos
df['duration'] = df['duration'].apply(lambda x: int(x.split()[0]) if isinstance(x, str) and 'min' in x else (int(x.split()[0]) * 60 if isinstance(x, str) else 0))

# Codificar a coluna 'type' para valores numéricos
label_encoder = LabelEncoder()
df['type'] = label_encoder.fit_transform(df['type'])

# Converter dados categóricos para numéricos usando one-hot encoding
df_encoded = pd.get_dummies(df, columns=['country', 'rating', 'listed_in'])

# Normalizar os dados
scaler = StandardScaler()
df_encoded[['release_year', 'duration']] = scaler.fit_transform(df_encoded[['release_year', 'duration']])

# Separar os dados em variáveis de características e alvo
X = df_encoded.drop(columns=['type'])
y = df_encoded['type']

# Separar os dados em conjuntos de treino e teste (90% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

# Função de aptidão para o algoritmo genético
def fitness_func(ga_instance, solution, solution_idx):
    # Extrair os parâmetros C e gamma da solução
    C = solution[0]
    gamma = solution[1]
    
    # Treinar o modelo SVM com os parâmetros fornecidos
    svm_model = SVC(C=C, gamma=gamma)
    svm_model.fit(X_train, y_train)
    
    # Fazer previsões no conjunto de teste
    y_pred = svm_model.predict(X_test)
    
    # Calcular a acurácia no conjunto de teste
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

# Definir a função de aptidão para o PYGAD
fitness_function = fitness_func

# Parâmetros do algoritmo genético
num_generations = 50
num_parents_mating = 5
sol_per_pop = 10
num_genes = 2

# Intervalo de valores para os parâmetros C e gamma
gene_space = [{'low': 0.1, 'high': 100}, {'low': 0.0001, 'high': 1}]

# Criar uma instância do algoritmo genético PYGAD
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       gene_space=gene_space)

# Executar o algoritmo genético
ga_instance.run()

# Obter a melhor solução encontrada pelo algoritmo genético
solution, solution_fitness, solution_idx = ga_instance.best_solution()

print(f"Melhor solução: C={solution[0]}, gamma={solution[1]}")
print(f"Acurácia da melhor solução: {solution_fitness}")

c:\Users\emanu\my-project\.venv\Lib\site-packages\pygad\pygad.py:728: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resulted in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(
c:\Users\emanu\my-project\.venv\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


Melhor solução: C=70.1175325240478, gamma=0.06835541674703072
Acurácia da melhor solução: 0.9771666456414785
